This notebook provides a prototype for a Genie Agent using the databricks ai_bridge repo: https://github.com/databricks/databricks-ai-bridge, as well as creating tools for Vector Search and UC Functions

In [0]:
%pip install databricks_langchain mlflow
%restart_python

## Genie Agent
Databricks LangChain (e.g. AI Bridge) has a wrapper around the Genie Conversation API that allows the creation of a LangGraph Node or Agent for a Genie Space.

In [0]:
from databricks_langchain.genie import GenieAgent
import mlflow

genie_space_id =  "01f018e761531cb190d829f8d347d48f"
genie_agent = GenieAgent(
  genie_space_id, 
  "marketing_agent", 
  description="""
  This agent can access information about marketing campaigns and their performance. Example questions include
  - How has the total number of emails sent, delivered, and the unique clicks evolved over the last six months?
  - Which industries have shown the highest engagement rates with marketing campaigns?
  """)

In [0]:
input_example = {
    "messages": [
        {
            "role": "user",
            "content": "What are the top 3 subject lines for my campaigns that led to the most number of opens?",
        },        
    ]
}

In [0]:
genie_agent.invoke(input_example)

## Vector Search
We can also serve Vector Search as either a Langchain Retriever, or a UC Function. My preference right now is the UC Function due to the universal usability in a) Playground, b) ReACT Agents, and c) Agents via tools.

In [0]:
%pip install databricks-vectorsearch
dbutils.library.restartPython()

In [0]:
from databricks.vector_search.client import VectorSearchClient
client = VectorSearchClient()

In [0]:
%sql
ALTER TABLE shm.marketing.campaigns 
SET TBLPROPERTIES (delta.enableChangeDataFeed = true)

In [0]:
source_table = "shm.marketing.campaigns"
index_name = source_table+"_index"
try:
  index = client.create_delta_sync_index(
    endpoint_name="one-env-shared-endpoint-3",
    source_table_name=source_table,
    index_name=index_name,
    pipeline_type="TRIGGERED",
    primary_key="campaign_id",
    embedding_source_column="campaign_description",
    embedding_model_endpoint_name="databricks-gte-large-en"
  )
except Exception as e:
  print(e)
  index = client.get_index(index_name=index_name)
  index.sync()

Once created, we can sync using the following cell

In [0]:
%sql
CREATE OR REPLACE FUNCTION shm.marketing.campaign_search(
  description STRING COMMENT 'A campaign description'
)
RETURNS TABLE (
  matching_descriptions STRING
)
COMMENT 'Returns the top three campaing descriptions based on semantically similarity to the input query.
'
RETURN
SELECT campaign_description
FROM vector_search(
  index=>'shm.marketing.campaigns_index',
  query_text=>description,
  num_results=>3,
  query_type=>'hybrid'
)

In [0]:
%sql
SELECT * FROM shm.marketing.campaigns

In [0]:
%sql
SELECT * 
FROM shm.marketing.campaign_search(
  'Toyota speakers'
  )